<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Extracting Base Estimators from Bagged Models

---

In this lab, you will have to make use of the attributes available with sklearn's [BaggingClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.BaggingClassifier.html). In particular
you will need to investigate what you can do with 
- `.base_estimator_`
- `.estimators_`
- `.estimators_samples_`
- `.estimators_features_`

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#1.-Load-the-breast-cancer-data." data-toc-modified-id="1.-Load-the-breast-cancer-data.-1">1. Load the breast cancer data.</a></span></li><li><span><a href="#2.-Load-required-sklearn-packages." data-toc-modified-id="2.-Load-required-sklearn-packages.-2">2. Load required sklearn packages.</a></span></li><li><span><a href="#3.-Make-a-train-test-split." data-toc-modified-id="3.-Make-a-train-test-split.-3">3. Make a train-test split.</a></span></li><li><span><a href="#4.-Create-and-fit-a-BaggingClassifier-with-a-DecisionTreeClassifier-base-estimator." data-toc-modified-id="4.-Create-and-fit-a-BaggingClassifier-with-a-DecisionTreeClassifier-base-estimator.-4">4. Create and fit a <code>BaggingClassifier</code> with a <code>DecisionTreeClassifier</code> base estimator.</a></span></li><li><span><a href="#5.-Pull-out-the-base-estimator-from-the-ensemble-model." data-toc-modified-id="5.-Pull-out-the-base-estimator-from-the-ensemble-model.-5">5. Pull out the base estimator from the ensemble model.</a></span></li><li><span><a href="#6.-Pull-out-all-the-base-estimators." data-toc-modified-id="6.-Pull-out-all-the-base-estimators.-6">6. Pull out <em>all</em> the base estimators.</a></span></li><li><span><a href="#7.-Get-the-features-used-in-each-of-the-bagged-base-estimators." data-toc-modified-id="7.-Get-the-features-used-in-each-of-the-bagged-base-estimators.-7">7. Get the features used in each of the bagged base estimators.</a></span></li><li><span><a href="#8.-Create-a-list-of-the-features-used-in-the-first-base-estimator." data-toc-modified-id="8.-Create-a-list-of-the-features-used-in-the-first-base-estimator.-8">8. Create a list of the features used in the first base estimator.</a></span></li><li><span><a href="#9.-Get-out-the-samples-used-in-our-first-base-estimator." data-toc-modified-id="9.-Get-out-the-samples-used-in-our-first-base-estimator.-9">9. Get out the samples used in our first base estimator.</a></span></li><li><span><a href="#10.-Get-out-the-target-subsample-for-the-estimator." data-toc-modified-id="10.-Get-out-the-target-subsample-for-the-estimator.-10">10. Get out the target subsample for the estimator.</a></span></li><li><span><a href="#11.-Fit-a-decision-tree-equivalent-to-our-first-base-estimator." data-toc-modified-id="11.-Fit-a-decision-tree-equivalent-to-our-first-base-estimator.-11">11. Fit a decision tree equivalent to our first base estimator.</a></span></li><li><span><a href="#12.-Bonus:-Take-each-of-the-decision-trees-from-the-ensemble-above-and-obtain-its-predictions-for-the-target-variable-in-the-test-set.-Use-majority-voting-to-obtain-the-ensemble-prediction-for-the-target-label.-Compare-with-the-bagging-classifier-score." data-toc-modified-id="12.-Bonus:-Take-each-of-the-decision-trees-from-the-ensemble-above-and-obtain-its-predictions-for-the-target-variable-in-the-test-set.-Use-majority-voting-to-obtain-the-ensemble-prediction-for-the-target-label.-Compare-with-the-bagging-classifier-score.-12">12. Bonus: Take each of the decision trees from the ensemble above and obtain its predictions for the target variable in the test set. Use majority voting to obtain the ensemble prediction for the target label. Compare with the bagging classifier score.</a></span></li></ul></div>

### 1. Load the breast cancer data.

In [1]:
import pandas as pd
import numpy as np
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()

# Converting data into a dataframe structure 
X = pd.DataFrame(data['data'], columns=data['feature_names'])
# Setting up our Y value as well
y = pd.Series(data['target'])

In [2]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [3]:
X.shape

(569, 30)

In [4]:
X.describe()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
count,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,...,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000,569.000000
mean,14.127292,19.289649,91.969033,654.889104,0.096360,0.104341,0.088799,0.048919,0.181162,0.062798,...,16.269190,25.677223,107.261213,880.583128,0.132369,0.254265,0.272188,0.114606,0.290076,0.083946
std,3.524049,4.301036,24.298981,351.914129,0.014064,0.052813,0.079720,0.038803,0.027414,0.007060,...,4.833242,6.146258,33.602542,569.356993,0.022832,0.157336,0.208624,0.065732,0.061867,0.018061
min,6.981000,9.710000,43.790000,143.500000,0.052630,0.019380,0.000000,0.000000,0.106000,0.049960,...,7.930000,12.020000,50.410000,185.200000,0.071170,0.027290,0.000000,0.000000,0.156500,0.055040
25%,11.700000,16.170000,75.170000,420.300000,0.086370,0.064920,0.029560,0.020310,0.161900,0.057700,...,13.010000,21.080000,84.110000,515.300000,0.116600,0.147200,0.114500,0.064930,0.250400,0.071460
50%,13.370000,18.840000,86.240000,551.100000,0.095870,0.092630,0.061540,0.033500,0.179200,0.061540,...,14.970000,25.410000,97.660000,686.500000,0.131300,0.211900,0.226700,0.099930,0.282200,0.080040
75%,15.780000,21.800000,104.100000,782.700000,0.105300,0.130400,0.130700,0.074000,0.195700,0.066120,...,18.790000,29.720000,125.400000,1084.000000,0.146000,0.339100,0.382900,0.161400,0.317900,0.092080
max,28.110000,39.280000,188.500000,2501.000000,0.163400,0.345400,0.426800,0.201200,0.304000,0.097440,...,36.040000,49.540000,251.200000,4254.000000,0.222600,1.058000,1.252000,0.291000,0.663800,0.207500


### 2. Load required sklearn packages.

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

### 3. Make a train-test split.

In [6]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

In [7]:
X = scaler.fit_transform(X)

In [8]:
from sklearn.model_selection import cross_val_score, train_test_split

In [9]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, shuffle=True, random_state=1)

### 4. Create and fit a `BaggingClassifier` with a `DecisionTreeClassifier` base estimator.

- Fit on the training data.
- Report the score on the test data.

In [10]:
dt = DecisionTreeClassifier()

In [11]:
print("DT CV training score:\t", 
      cross_val_score(dt, X_train, y_train, cv=5,
                    n_jobs=1).mean())
dt.fit(X_train, y_train)
print("DT test score:\t", dt.score(X_test, y_test))

DT CV training score:	 0.9223734177215188
DT test score:	 0.9473684210526315


In [12]:
bagging = BaggingClassifier(base_estimator=dt,
                            max_samples=0.5, 
                            max_features=0.5, 
                            n_estimators=100)

print("DT Bagging CV training score:\t", 
      cross_val_score(bagging, X_train, y_train,
                    cv=5, n_jobs=1).mean())

bagging.fit(X_train, y_train)
print("DT bagging test score:\t", bagging.score(X_test, y_test))

DT Bagging CV training score:	 0.9498734177215189
DT bagging test score:	 0.9590643274853801


In [13]:
bagging = BaggingClassifier(base_estimator=dt,
                            max_samples=1., 
                            max_features=1., 
                            n_estimators=100)

print("DT Bagging CV training score:\t", 
      cross_val_score(bagging, X_train, y_train,
                    cv=5, n_jobs=1).mean())

bagging.fit(X_train, y_train)
print("DT bagging test score:\t", bagging.score(X_test, y_test))

DT Bagging CV training score:	 0.957373417721519
DT bagging test score:	 0.9590643274853801


### 5. Pull out the base estimator from the ensemble model.

.base_estimator_
.estimators_
.estimators_samples_
.estimators_features_

In [14]:
bagging.estimators_

[DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=424332229, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fraction_leaf=0.0, presort=False,
             random_state=1914749643, splitter='best'),
 DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
             max_features=None, max_leaf_nodes=None,
             min_impurity_decrease=0.0, min_impurity_split=None,
             min_samples_leaf=1, min_samples_split=2,
             min_weight_fr

### 6. Pull out *all* the base estimators.

In [15]:
bagging.base_estimator_

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

### 7. Get the features used in each of the bagged base estimators.

.base_estimator_
.estimators_
.estimators_samples_
.estimators_features_

In [40]:
bagging.estimators_features_

[array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]),
 array([ 0,  1,  2,  3,  4,  5,  6

### 8. Create a list of the features used in the first base estimator.

In [41]:
bagging.estimators_[0]

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False,
            random_state=424332229, splitter='best')

### 9. Get out the samples used in our first base estimator.

In [42]:
bagging.estimators_samples_[0]
len(bagging.estimators_samples_[0])

398

### 10. Get out the target subsample for the estimator.

In [39]:
y_train.iloc[bagging.estimators_samples_[0]].head()

412    1
427    1
144    1
64     0
368    0
dtype: int64

### 11. Fit a decision tree equivalent to our first base estimator.

In [24]:
X = pd.DataFrame(data['data'], columns=data['feature_names'])

In [26]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [27]:
X.columns

Index(['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry', 'worst fractal dimension'],
      dtype='object')

In [28]:
X_small = X.loc[ bagging.estimators_samples_[0],['mean radius', 'mean texture', 'mean perimeter', 'mean area',
       'mean smoothness', 'mean compactness', 'mean concavity',
       'mean concave points', 'mean symmetry', 'mean fractal dimension',
       'radius error', 'texture error', 'perimeter error', 'area error',
       'smoothness error', 'compactness error', 'concavity error',
       'concave points error', 'symmetry error', 'fractal dimension error',
       'worst radius', 'worst texture', 'worst perimeter', 'worst area',
       'worst smoothness', 'worst compactness', 'worst concavity',
       'worst concave points', 'worst symmetry']  ]

In [29]:
X_small

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,fractal dimension error,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry
107,12.360,18.54,79.01,466.7,0.08477,0.06815,0.026430,0.019210,0.1602,0.06066,...,0.001356,13.290,27.49,85.56,544.1,0.11840,0.19630,0.193700,0.084420,0.2983
74,12.310,16.52,79.19,470.9,0.09172,0.06829,0.033720,0.022720,0.1720,0.05914,...,0.002304,14.110,23.21,89.71,611.1,0.11760,0.18430,0.170300,0.086600,0.2618
301,12.460,19.89,80.43,471.3,0.08451,0.10140,0.068300,0.030990,0.1781,0.06249,...,0.004651,13.460,23.07,88.13,551.3,0.10500,0.21580,0.190400,0.076250,0.2685
37,13.030,18.42,82.61,523.8,0.08983,0.03766,0.025620,0.029230,0.1467,0.05863,...,0.001777,13.300,22.81,84.46,545.9,0.09701,0.04619,0.048330,0.050130,0.1987
27,18.610,20.25,122.10,1094.0,0.09440,0.10660,0.149000,0.077310,0.1697,0.05699,...,0.004217,21.310,27.26,139.90,1403.0,0.13380,0.21170,0.344600,0.149000,0.2341
163,12.340,22.22,79.85,464.5,0.10120,0.10150,0.053700,0.028220,0.1551,0.06761,...,0.005348,13.580,28.68,87.36,553.0,0.14520,0.23380,0.168800,0.081940,0.2268
245,10.480,19.86,66.72,337.7,0.10700,0.05971,0.048310,0.030700,0.1737,0.06440,...,0.003560,11.480,29.46,73.68,402.8,0.15150,0.10260,0.118100,0.067360,0.2883
119,17.950,20.01,114.20,982.0,0.08402,0.06722,0.072930,0.055960,0.2129,0.05025,...,0.001902,20.580,27.83,129.20,1261.0,0.10720,0.12020,0.224900,0.118500,0.4882
181,21.090,26.57,142.70,1311.0,0.11410,0.28320,0.248700,0.149600,0.2395,0.07398,...,0.005295,26.680,33.48,176.50,2089.0,0.14910,0.75840,0.678000,0.290300,0.4098
20,13.080,15.71,85.63,520.0,0.10750,0.12700,0.045680,0.031100,0.1967,0.06811,...,0.002425,14.500,20.49,96.09,630.5,0.13120,0.27760,0.189000,0.072830,0.3184


In [34]:
ys = y_train.iloc[bagging.estimators_samples_[0]]

In [35]:
Xs = scaler.fit_transform(X_small)

In [36]:
Xs_train, Xs_test, ys_train, ys_test = train_test_split(
    Xs, ys, test_size=0.3, stratify=ys, shuffle=True, random_state=1)

In [37]:
dt_s = DecisionTreeClassifier()

In [38]:
print("DT CV training score:\t", 
      cross_val_score(dt_s, Xs_train, ys_train, cv=5,
                    n_jobs=1).mean())
dt_s.fit(Xs_train, ys_train)
print("DT test score:\t", dt_s.score(Xs_test, ys_test))

DT CV training score:	 0.6372077922077921
DT test score:	 0.6916666666666667


### 12. Bonus: Take each of the decision trees from the ensemble above and obtain its predictions for the target variable in the test set. Use majority voting to obtain the ensemble prediction for the target label. Compare with the bagging classifier score.